# Working with Noise Covariance Matrices 

This notebook demonstrates how to attach noise covariance matrices to the Network class objects and manipulating those Networks with algebra.

## Example 1: Cascading Two Resistive Two-Port Networks

In this example a shunted resistor is cascaded with a series resistor

In [2]:
import skrf as rf
import numpy as npy
from skrf.constants import *

T0 = 290. # Both resistors are set at room temperature for now
frequency = rf.Frequency(start=1000, stop=2000, npoints=10, unit='MHz')

ovec = npy.ones(len(frequency))
zvec = npy.zeros(len(frequency))
rseries = 200*ovec # Series resistance
rshunt = 500*ovec  # Shunt resistance

### shunted resistor two port ###
# This is the Z form of the network parameters for a shunted resistor, the rf.network_array(...) function places the array of vectors into a form the Network object will work with.
r_shunt_z = rf.network_array([[rshunt, rshunt],
                              [rshunt, rshunt]])

# This is the Cz form of the covariance matrix for a shunted resistor
r_shunt_z_cov = 4.*K_BOLTZMANN*T0*npy.real(r_shunt_z)

# This is the Y form of the network parameters for a series resistor
r_series_y = rf.network_array([[1/rseries,  -1/rseries],
                               [-1/rseries,  1/rseries]])

# This is the Cy form of the covariance matrix for a series resistor
r_series_y_cov = 4.*K_BOLTZMANN*T0*npy.real(r_series_y)

# The two functions from_y and from_z within the Network class are used to create instances of the object using the network representations above  
ntwkA = rf.Network.from_z(r_shunt_z, frequency = frequency)
ntwkB = rf.Network.from_y(r_series_y, frequency = frequency)

# NetworkNoiseCov objects contain the covariance matricies that will be associated with the Network objects above. 
nc_rseries = rf.NetworkNoiseCov(r_series_y_cov, form='y')
nc_rshunt = rf.NetworkNoiseCov(r_shunt_z_cov, form='z')

# You attach a NetworkNoiseCov object to the network with the noise_source(...) function
ntwkA.noise_source(nc_rshunt)
ntwkB.noise_source(nc_rseries)

# Here we assume the two networks are two-port networks and cascade the two together to make a combined two-port network called ntwkC
ntwkC = rf.cascade_2port(ntwkA, ntwkB)

# The noise paramters of the combined network can then be calculated via a number of properties
ntwkC.nfmin        # Minimum noise factor (linear)
ntwkC.y_opt        # Optimal source admittance
ntwkC.z_opt        # Optimal source impedance
ntwkC.rn           # Scaling factor
ntwkC.nf(50)       # Noise factor for a given source impedance


array([5.94, 5.94, 5.94, 5.94, 5.94, 5.94, 5.94, 5.94, 5.94, 5.94])

In [3]:

# The above example can be made a little cleaner when we know the networks are passive. Here again we create the two Networks
r_shunt_z = rf.network_array([[rshunt, rshunt],
                              [rshunt, rshunt]])

r_series_y = rf.network_array([[1/rseries,  -1/rseries],
                               [-1/rseries,  1/rseries]])

ntwkA = rf.Network.from_z(r_shunt_z, frequency = frequency)
ntwkB = rf.Network.from_y(r_series_y, frequency = frequency)

# This time however, we will simply tell the networks that they are passive components. They will set the NetworkNoiseCov themselves using k*T0*(1-SS^*)/2
ntwkA.noise_source('passive')

# You can change the temperature of the Network using this function as well (the temperature defaults to T=290 [K])
ntwkB.noise_source('passive', T0=150)

# Cascade the two networks
ntwkC = rf.cascade_2port(ntwkA, ntwkB)

# The noise paramters of the combined network can then be calculated via a number of properties
ntwkC.nfmin        # Minimum noise factor (linear)
ntwkC.y_opt        # Optimal source admittance
ntwkC.z_opt        # Optimal source impedance
ntwkC.rn           # Scaling factor
ntwkC.nf(50)       # Noise factor for a given source impedance



array([3.60344828, 3.60344828, 3.60344828, 3.60344828, 3.60344828,
       3.60344828, 3.60344828, 3.60344828, 3.60344828, 3.60344828])

## Example 2: Cascading Two Matched Attenuators

In this example two attenuators are cascaded to test the Friis transmission formula for Noise Figure

In [1]:
import skrf as rf
import numpy as npy
from skrf.constants import *

# Testing cascade
T0 = 290
atten1 = 10**(-1/20)  # Attenuation of first attenuator
atten2 = 10**(-4/20)  # Attenuation of second attenuator

frequency = rf.Frequency(start=1000, stop=2000, npoints=10, unit='MHz')

ovec = npy.ones(len(frequency))
zvec = npy.zeros(len(frequency))

attn1_vec = rf.network_array([[zvec,    atten1*ovec],
                              [atten1*ovec,   zvec]])

attn2_vec = rf.network_array([[zvec,    atten2*ovec],
                              [atten2*ovec,   zvec]])

ntwkA = rf.Network(s=attn1_vec, frequency=frequency)
ntwkB = rf.Network(s=attn2_vec, frequency=frequency)

# Since both networks are passive, we do as we did above 
ntwkA.noise_source('passive')
ntwkB.noise_source('passive')

ntwkC = rf.cascade_2port(ntwkA, ntwkB)
a = ntwkC.nf_db(50)
print(a)
# The result for NF is simply the sum of the two attenuators



[5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]


In [4]:
# Annnnd even easier, but less clear
ntwkAttn1 = rf.components.Attenuator(5, frequency=frequency)
ntwkAttn2 = rf.components.Attenuator(7, frequency=frequency)
ntwkC = rf.cascade_2port(ntwkAttn1, ntwkAttn2)
a = ntwkC.nf_db(50)
print(a)

# The components package contains common noisy elements


[12. 12. 12. 12. 12. 12. 12. 12. 12. 12.]
